In [1]:
#Scrapping and organizing data from Wikipedia

In [102]:
import pandas as pd

In [103]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = data[0] #creating a variable only with the desired table
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [104]:
#dropping not assigned in Borough 
df = df[df.Borough!= 'Not assigned'] 

#making sure to not have 'Not assigned' in Neighbourhood
df = df.reset_index(drop=True)
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [105]:
df.shape

(103, 3)

## Part 2

In [107]:
#Getting data from csv file since geocoder didn't work
data_excel = pd.read_csv('http://cocl.us/Geospatial_data')
data_excel.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [108]:
#Merging both data frames based on Postal Code
df_space = pd.merge(df, data_excel, on=['Postal Code'], how='outer')
df_space.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Part 3

In [109]:
import geopy
from geopy.geocoders import Nominatim
#!conda install -c conda-forge folium=0.5.0 --yes
#conda install --override-channels -c main package_in_main
import folium

In [110]:
#Getting the coordinates of Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toront are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toront are 43.6534817, -79.3839347.


In [117]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_space['Latitude'], df_space['Longitude'], df_space['Borough'], df_space['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto